In [1]:
import imp
import os
import cv2
import pdb
import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
from multiprocessing import Pool
import torch
from torch import optim
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
# from torch.nn.functional import InterpolationMode
import torchvision.models as models
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import logging 
import datetime
import sys
import cv2
import matplotlib.pyplot as plt
import PIL

In [2]:
gloss_df_path = "data_validation/processed_gloss.csv"
gloss_df = pd.read_csv(gloss_df_path)
gloss_df.dropna(inplace=True)
gloss_df.replace(to_replace="ASHAG", value="AŞAĞI", inplace=True)
gloss_df['glossRange'] = gloss_df['glossEnd'] - gloss_df['glossStart']
# gloss_df.head()

In [3]:
num_classes = len(gloss_df.gloss.unique())

In [4]:
import torchviz
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler
from torchvision.models import squeezenet1_1, SqueezeNet1_1_Weights
from torchvision import transforms

In [5]:
from pytorchvideo.data import labeled_video_dataset

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    UniformTemporalSubsample,
    Permute
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomAdjustSharpness,
    Resize
)

from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo
)

/home/toghrul/anaconda3/envs/sign-lang/lib/python3.9/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/toghrul/anaconda3/envs/sign-lang/lib/python3.9/site-packages/torchvision/transforms/_transforms_video.py:25: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [6]:
video_transform = Compose([
    ApplyTransformToKey(key="video",
    transform=Compose([
        UniformTemporalSubsample(25),
        Lambda(lambda x: x/255),
        Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
        RandomShortSideScale(min_size=224, max_size=256),
        CenterCropVideo(224),
    ]),
    ),
])

In [7]:
train_path = "/home/toghrul/SLR/data/train"
val_path = "/home/toghrul/SLR/data/val"
test_path = "/home/toghrul/SLR/data/test"

In [8]:
from pytorch_lightning import LightningModule, seed_everything, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import classification_report
import torchmetrics

In [9]:
# video_model = torch.hub.load('facebookresearch/pytorchvideo', 'efficient_x3d_xs', pretrained=True)

In [10]:
# a = torch.randn(size=(8, num_classes)).requires_grad_()
# b = torch.randint(5, size=(8, ), dtype=torch.long)
# a, b

In [11]:
# b = F.one_hot(b, num_classes=num_classes).float()

In [12]:
# criterion = nn.BCEWithLogitsLoss()
# criterion(a, b)

In [13]:
# loss = nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.empty(3, dtype=torch.long).random_(5)
# output = loss(input, target)
# output

In [14]:
# input, target

In [31]:
class VideoModel(LightningModule):
    def __init__(self, ):
        super(VideoModel, self).__init__()

        self.video_model = torch.hub.load('facebookresearch/pytorchvideo', 'efficient_x3d_xs', pretrained=True)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(400, num_classes)

        self.lr = 1e-3
        self.batch_size = 8
        self.num_worker = 4
        self.num_steps_train = 0
        self.num_steps_val = 0

        self.metric = torchmetrics.classification.MultilabelAccuracy(num_labels=num_classes)

        #loss
        self.criterion = nn.BCEWithLogitsLoss()

    def forward(self, x):
        x = self.video_model(x)
        x = self.relu(x)
        x = self.fc(x)
        return x

    def configure_optimizers(self):
        opt = torch.optim.AdamW(params=self.parameters(), lr = self.lr)
        scheduler = CosineAnnealingLR(opt, T_max=10, eta_min=1e-6, last_epoch=-1)
        return {'optimizer': opt,
                'lr_scheduler': scheduler}

    def train_dataloader(self):
        dataset = labeled_video_dataset(train_path, clip_sampler=make_clip_sampler('random', 2),
                                         transform=video_transform, decode_audio=False)

        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.num_worker, pin_memory=True)

        return loader

    def training_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        label = F.one_hot(label, num_classes=num_classes).float()
        out = self(video)
        print(f">>> Training step No.{self.num_steps_train}:")
        # print("Pred:", out)
        # print("GT:", label)
        # print(f"Pred:\n{out}")
        # print(f"Pred shape:\n{out.shape}")
        # print(f"Label:\n{label}")
        # print(f"Label shape:\n{label.shape}")
        # print(">>> INFO: Computing Training Loss")
        loss = self.criterion(out, label)
        print(f"Loss: {loss}")
        self.num_steps_train += 1
        # print(">>> INFO: Training Loss Computed")
        # print(">>> INFO: Computing Training Metric")
        metric = self.metric(out, label)
        print(f"Accuracy: {metric}")

        values = {"loss": loss,
                "metric": metric.detach()}
        
        self.log_dict({"step_loss": loss,
                        "step_metric": metric.detach()})
        
        return values
        
        # return {"loss": loss}

    def training_epoch_end(self, outputs):
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().numpy().round(2)
        
        self.log('training_loss', loss)
        print(f">>> Epoch end loss: {loss}")
        self.log('training_metric', metric)
        

    def val_dataloader(self):
        dataset = labeled_video_dataset(val_path, clip_sampler=make_clip_sampler('random', 2),
                                         transform=video_transform, decode_audio=False)

        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.num_worker, pin_memory=True)

        return loader

    def validation_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        label = F.one_hot(label, num_classes=num_classes).float()
        out = self(video)
        # print(">>> INFO: Computing Val Loss")
        print(f">>> Validation step No.{self.num_steps_val}:")
        loss = self.criterion(out, label)
        print(f"Loss: {loss}")
        self.num_steps_val += 1
        # print(">>> INFO: Val Loss Computed")
        # print(">>> INFO: Computing Val Metric")
        metric = self.metric(out, label)
        print(f"Accuracy: {metric}")
        

        return {"loss": loss,
                "metric": metric.detach()}
        
        # return {"loss": loss}

    def validation_epoch_end(self, outputs):
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().numpy().round(2)
        self.log('val_loss', loss)
        self.log('val_metric', metric)

    def test_dataloader(self):
        dataset = labeled_video_dataset(test_path, clip_sampler=make_clip_sampler('random', 2),
                                         transform=video_transform, decode_audio=False)

        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.num_worker, pin_memory=True)

        return loader

    def test_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        label = F.one_hot(label, num_classes=num_classes).float()
        out = self.forward(video)
        metric = self.metric(out, label)

        return {"label": label,
                "pred": out.detach(),}

    def test_epoch_end(self, outputs):
        label=torch.cat([x['label'] for x in outputs]).cpu().numpy()
        pred = torch.cat([x['pred'] for x in outputs]).cpu().numpy()

        print(f">> Label: {label}\nPred: {pred}")




In [16]:
checkpoint_callback = ModelCheckpoint(monitor='val_loss', dirpath="checkpoints", 
                                    verbose=True, save_last=True, save_top_k=2)
lr_monitor = LearningRateMonitor(logging_interval="epoch")


In [32]:
model = VideoModel()
seed_everything(0)

trainer = Trainer(max_epochs=10,
                accelerator="gpu", devices=-1,
                precision=16,
                # accumulate_grad_batches=2,
                enable_progress_bar=True,
                # num_sanity_val_steps=0,
                callbacks=[lr_monitor, checkpoint_callback],
                log_every_n_steps=5,
                limit_train_batches=10,
                limit_val_batches=5,
                limit_test_batches=5,)

Using cache found in /home/toghrul/.cache/torch/hub/facebookresearch_pytorchvideo_main
Global seed set to 0
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [19]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type               | Params
---------------------------------------------------
0 | video_model | EfficientX3d       | 3.8 M 
1 | relu        | ReLU               | 0     
2 | fc          | Linear             | 93.8 K
3 | metric      | MultilabelAccuracy | 0     
4 | criterion   | BCEWithLogitsLoss  | 0     
---------------------------------------------------
3.9 M     Trainable params
0         Non-trainable params
3.9 M     Total params
7.776     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

>>> Validation step No.0:
Loss: 0.7293049693107605
Accuracy: 0.5438034534454346
>>> Validation step No.1:
Loss: 0.7303302884101868
Accuracy: 0.5507479310035706


Training: 0it [00:00, ?it/s]

>>> Training step No.0:
Loss: 0.7077091932296753
Accuracy: 0.5464743971824646
>>> Training step No.1:
Loss: 0.5746216177940369
Accuracy: 0.7393162846565247
>>> Training step No.2:
Loss: 0.3870113492012024
Accuracy: 0.8616453409194946
>>> Training step No.3:
Loss: 0.2158113569021225
Accuracy: 0.939636766910553
>>> Training step No.4:
Loss: 0.10342903435230255
Accuracy: 0.9780983328819275
>>> Training step No.5:
Loss: 0.056837841868400574
Accuracy: 0.9914530515670776
>>> Training step No.6:
Loss: 0.03889686241745949
Accuracy: 0.9957265257835388
>>> Training step No.7:
Loss: 0.040093887597322464
Accuracy: 0.9957265257835388
>>> Training step No.8:
Loss: 0.04646025225520134
Accuracy: 0.9957265257835388
>>> Training step No.9:
Loss: 0.04310651496052742
Accuracy: 0.9957265853881836


...

Validation: 0it [00:00, ?it/s]

>>> Validation step No.47:
Loss: 0.031411007046699524
Accuracy: 0.9957265257835388
>>> Validation step No.48:
Loss: 0.031880270689725876
Accuracy: 0.995192289352417
>>> Validation step No.49:
Loss: 0.03245586156845093
Accuracy: 0.9946581125259399
>>> Validation step No.50:
Loss: 0.02989715337753296
Accuracy: 0.9957265257835388


Epoch 9, global step 100: 'val_loss' was not in top 2


>>> Validation step No.51:
Loss: 0.031822048127651215
Accuracy: 0.995192289352417
>>> Epoch end loss: 0.019999999552965164


`Trainer.fit` stopped: `max_epochs=10` reached.


In [29]:
model = VideoModel.load_from_checkpoint(
    checkpoint_path="/home/toghrul/SLR/sign-lang/checkpoints/last.ckpt",
    hparams_file="/home/toghrul/SLR/sign-lang/lightning_logs/version_38/hparams.yaml",
    map_location=None,
)

Using cache found in /home/toghrul/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [33]:
trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

>> Label: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Pred: [[-0.4612    0.2253    0.001697 ...  0.693    -0.00444   0.1227  ]
 [-0.2053    0.342     0.7627   ...  0.6035   -0.3425   -0.04852 ]
 [ 0.3962    0.5854    0.5273   ...  0.7075   -0.0956    0.3489  ]
 ...
 [-0.369     0.71      0.2156   ... -0.07446   0.1896    0.6064  ]
 [-0.584     0.3564    0.356    ...  0.28      0.14     -0.3337  ]
 [-0.3782    0.6465    0.548    ...  0.525    -0.1487    0.0789  ]]


[{}]